325 Final Project: Nathaniel Gilbert, Emile Rebillard, Thomas Sorensen, Angelina Ponce

In [1]:
import pandas as pd

Introduction

Our position are Data Analysts hired by Harvard University to analyze the key drivers of Academic Performance for NCAA Division I athletes and exploring methods to enhance it. We are going to focus on the Academic Progress Rate (APR), which is a calculation based off how many athletes on a team remain academiclly elligible and with the university. Most of the data within the NCAA dataset is not gathered by the NCAA. In fact, almost all of it is gathered by the member schools, and then shared with the NCAA. However, some of this data has been directly gathered by the NCAA.

In [ ]:
raw_data = pd.read_csv('C:/Users/natha/OneDrive/Documents/GitHub/DIDA_325_Final-project/csvs/raw data.csv')
drops=[0,12,20,85]
raw_data.drop(raw_data.columns[drops], axis=1, inplace=True)
# These columns were dropped because they were entirely empty.

cols_to_fill = list(range(82, 97)) # the columns we want to fill are the 83rd to 98th columns (0-indexed) so we are creating a list of their indices using the range function. 
raw_data.iloc[:, cols_to_fill] = raw_data.iloc[:, cols_to_fill].fillna(0)
# These columns were filled with 0s because some were empty but the documnetation said they should be 0s.

The columns remaining are SCL_UNITID (int) which is the id for each column, SCL_NAME (character), the name for the university, Sport_Code (Cateogry) and Sport_Name (chategory) which represent the id and name of the respective sport, Academic_Year(category) the year for that row, SCL_DIV_19 (Category), and SCL_SUB_19 (Category) which represent the NCAA division and subdivision they play in. D1_FB_CONF (character) and CONFNAME (character) which are the schools football and regular conference respectivley. SL_HBCU (category) and SL_Private (category) which is a binary data that shows if a school is HBCU and if a school is private. Then there are MULTIYR_APR_RATE_1000_RAW, MULTIYR_APR_RATE_1000_CI, MULTIYR_APR_RATE_1000_OFFICIAL, RAW_OR_CI, MULTIYR_SQUAD_SIZE, MULTIYR_ELIG_RATE,MULTIYR_RET_RATE which are all numeric and represent multiyear academic stats. Then from each year from 2006-2019 there are APR_RATE_XXXX_1000	ELIG_RATE_XXXX	RET_RATE_XXX	NUM_OF_ATHLETES_XXXX which are all numeric and represent single year academic stats. Finally from 2006-2020 there are PUB_AWARD_XX which are categorical values that display if a school got an academic award from the NCAA.

Next, using the cleaned dataset we will be looking at four different research questions and attempting to answer them by analyzing the dataset.

1. How have the retention rates for NCAA schools changed over the available data, from 2004 to 2019? We will be using all of the RET_Rate_XXXX columns where XXXX represents a different year for each column
2. Exploratory analysis: How many sports are fielded by each institution, and does that vary between NCAA divisions?  We will be using the SCL_UNITID and SCL_NAME columns as well as the Sport_Code and Sport_Name to analyse the counts of sports by each university. This is important to see if schools generally field the same number of sports or if there's drastic differences between schools and analyze what could be different from schools with a high number of sports and those with a low number.
3. What variables are the most positively correlated with multi-year APR, and can we accurately predict what a teams multi-year APR will be given other data? (regression). This will involve using linear regression on the whole dataset to determine which variables are significant.
4. Do Private or Public schools have better APR and academic eligibility rates? We will be using the multi-year APR and multi-year academic eligibility columns and SCL_PRIVATE column.
